In [1]:
import numpy as np
import awkward as ak
import uproot
import matplotlib.pyplot as plt
from os import path
import hist
from hist import Hist
import mplhep
import json
from matplotlib.patches import Rectangle

#PRESEL DATA (NEEDS X,Y, and chi2 track information!)
analyzed_data_path = "/home/users/hswanson13/tbanalysis/output_analysis_data/with_tracks_DESY_ex/start_5050_stop_5199_bias_260_offset_20.0_energy_5.0_power_i1_file_from_DESY_MERGED_module_36_LP2_20_cubicLM_unbinned_add_tracks/"
dataPresel = ak.from_parquet(
    path.join(
        analyzed_data_path, 
        "fullPresel.parquet"
    )
)

In [2]:
def find_pxl_from_traks(xtracks, ytracks, eps=0.23, min_samples=20):
    from sklearn.cluster import DBSCAN
    """
    xtracks, ytracks = tracker hit coordinate
    At a distance "eps" from the point xtrack,ytrack, it should contain "min_samples", this defines the density, if it is failing play with these numbers
    """
    xtracks = np.array(xtracks) #just in case not numpy arrays
    ytracks = np.array(ytracks)
    if not xtracks.size or not ytracks.size:
        return []
    elif xtracks is None or ytracks is None:
        raise ValueError("HEY these cant be None, should be empty list, you probably filled the array incorrectly!")
    
    points = np.column_stack((xtracks, ytracks))
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(points) #WARNING! very slow algorithm, O(n^2) but very good at getting center
    labels = db.labels_ #-1 are noise points
    grouped_points = {}
    for lbl in np.unique(labels):
        #group labeled points
        grouped_points[lbl] = points[np.where(labels == lbl)]
    
    found_groups = []
    for gp, gpnts in grouped_points.items():
        if gp != -1: #if -1 it is noise!
            xcenter = np.mean(gpnts[:,0])
            ycenter = np.mean(gpnts[:,1])
            width = abs(gpnts[:,0].max() - gpnts[:,0].min())
            height = abs(gpnts[:,1].max() - gpnts[:,1].min())
            found_groups.append(
                {
                    'xcenter': xcenter,
                    'ycenter': ycenter,
                    'width': width,
                    'height': height
                }
            )
    return found_groups

def get_xy_hits(data_presel, nhits=None, equals_nhits=False):
    xhits = np.empty((16,16), dtype=object)
    yhits = np.empty((16,16), dtype=object)

    chi2_sel = ((data_presel.chi2 > 0) & (data_presel.chi2 < 30))
    bad_track = ((data_presel['x'] != -1000) | (data_presel['y'] != -1000))
    for row in range(16):
        for col in range(16):
            pix_sel = ak.any(((data_presel.row==row)&(data_presel.col==col)), axis=-1) #ak.any to grab track for event that had a hit in the row or col
            total_sel = pix_sel & chi2_sel & bad_track
            if nhits is not None:
                _nhits = ak.count(data_presel.row, axis=-1)
                if equals_nhits:
                    total_sel = total_sel & (_nhits == nhits)
                else:
                    total_sel = total_sel & (_nhits > nhits)
            xtracks = ak.to_numpy(ak.flatten(data_presel[total_sel].x))
            ytracks = ak.to_numpy(ak.flatten(data_presel[total_sel].y))
            xhits[row,col] = np.append(xhits[row,col], xtracks) if xhits[row, col] is not None else xtracks
            yhits[row,col] = np.append(yhits[row,col], ytracks) if yhits[row, col] is not None else ytracks
    return xhits, yhits

In [3]:
#Get x,y hit data
#all hits
xhits, yhits = get_xy_hits(dataPresel)
#nhits greater than 1
xhits1, yhits1 = get_xy_hits(dataPresel, nhits=1)
#nhits equals 1
xhitse1, yhitse1 = get_xy_hits(dataPresel, nhits=1, equals_nhits=True)

In [ ]:
#Display plots of the x,y hit data as well as hit map
fig, axs = plt.subplots(2,2, figsize=(20,20), gridspec_kw = {'wspace':0.1, 'hspace':0.1})
sel_row, sel_col = 9, 8

for row in range(16):
    for col in range(16):
        #just selecting a group of pixels to label
        in_pix_range = (6<=row<=10 and 6<=col<=10)
        label = f"({row},{col})" if in_pix_range else ''
        marker = 'x' if in_pix_range else 'o'
        s=1 if in_pix_range else 1

        #fill all hits plot
        axs[0,0].scatter(xhits[row,col], yhits[row,col], s=s, label=label, marker=marker)
        #fill nhits>1 plot
        axs[1,0].scatter(xhits1[row,col], yhits1[row,col],s=s, label=label,  marker=marker)
        #do single pixel plot
        if row == sel_row and col == sel_col:
            px_pos = find_pxl_from_traks(xhits[row, col], yhits[row, col])[0]
            xcenter, ycenter = px_pos['xcenter'], px_pos['ycenter']
            #calculate ratio in and out for both
            in_sel = lambda xarr,yarr:  (
                (xcenter-1.3/2 <=xarr) & (xarr<=xcenter+1.3/2)
                &
                (ycenter-1.3/2 <=yarr) & (yarr<=ycenter+1.3/2)    
            )
            px_xhitse1, px_yhitse1 = xhitse1[row,col], yhitse1[row,col]
            px_xhits1, px_yhits1 = xhits1[row,col], yhits1[row,col]

            nhitse1_ratio = len(px_xhitse1[(np.logical_not(in_sel(px_xhitse1, px_yhitse1)))]) / len(px_xhitse1[in_sel(px_xhitse1, px_yhitse1)])
            nhits1_ratio = len(px_xhits1[(np.logical_not(in_sel(px_xhits1, px_yhits1)))]) / len(px_xhits1[in_sel(px_xhits1, px_yhits1)])

            axs[1,1].scatter(px_xhitse1, px_yhitse1,s=s, label=f"nhits==1, r={nhitse1_ratio:.3}",  marker=marker, c='blue')
            axs[1,1].scatter(px_xhits1, px_yhits1,s=s, label=f"nhits>1. r={nhits1_ratio:.3}",  marker=marker, c='orange')
            axs[1,1].add_patch(Rectangle(xy=(xcenter-1.3/2, ycenter-1.3/2), width=1.3, height=1.3, linewidth=3, color='green', fill=False))

ticks = np.arange(-10,10+1,1)

axs[0,0].set_title("All Track Hits, colored by pixel")
axs[0,0].set_xticks(ticks)
axs[0,0].set_yticks(ticks)
axs[0,0].set_xlabel("x (mm)")
axs[0,0].set_ylabel("y (mm)")
lgnd = axs[0,0].legend(loc="upper right", scatterpoints=1, fontsize=10)
for handle in lgnd.legend_handles:
    handle.set_sizes([30.0])

axs[1,0].set_title("Tracks For Multi Hit Events")
axs[1,0].set_xticks(ticks)
axs[1,0].set_yticks(ticks)
axs[1,0].set_xlabel("x (mm)")
axs[1,0].set_ylabel("y (mm)")
lgnd = axs[1,0].legend(loc="upper right", scatterpoints=1, fontsize=10)
for handle in lgnd.legend_handles:
    handle.set_sizes([30.0])

axs[1,1].set_title(f"Pixel {sel_row,sel_col} Noise Comparison, r=not_in / in")
axs[1,1].set_xticks(ticks)
axs[1,1].set_yticks(ticks)
axs[1,1].set_xlabel("x (mm)")
axs[1,1].set_ylabel("y (mm)")
lgnd = axs[1,1].legend(loc="upper right", scatterpoints=1, fontsize=10)
for handle in lgnd.legend_handles:
    handle.set_sizes([30.0])

hitmap_hist = Hist(
    #x axis = col
    hist.axis.Integer(0, 16, name="col", label="col", flow=False),
    #y axis = row
    hist.axis.Integer(0, 16, name="row", label="row", flow=False),
)
hit_map_path = "/home/users/hswanson13/tbanalysis/output_analysis_data/checking_res_nhits/start_5050_stop_5199_bias_260_offset_20.0_energy_5.0_power_i1_file_from_DESY_module_36_LP2_20_cubicLM_unbinned/"
with open(path.join(hit_map_path, "hit_map.json"), "r") as f:
    hit_map = np.array(json.load(f))

for (row, col), weight in np.ndenumerate(hit_map):
    hitmap_hist.fill(row=row, col=col, weight=weight)
    # x = col, y = row
    axs[0,1].text(col + 0.5, row + 0.5, f'{int(weight)}', ha='center', va='center', color='w', fontsize="xx-small")
mplhep.hist2dplot(hitmap_hist, ax=axs[0,1], cmax=2300)
axs[0,1].set_title('Hit Map Histogram')

In [ ]:
#ONLY NEEDS DATAPRESEL WITH TRACK DATA!!!!!!!!!
def dt_plot(ax, dts, bins, sigma, colors: tuple, label=''):
    """
    Fits gaussian to dt distrobution and plots it
    """
    from scipy.optimize import curve_fit

    dt_hist, dt_bins = np.histogram(dts, bins)
    bin_centers = (dt_bins[:-1] + dt_bins[1:]) / 2
    ax.hist(dts, bins=dt_bins, color=colors[0], histtype='step')
    
    amplitude = np.max(dt_hist)
    peak_center = bin_centers[dt_hist == amplitude][0]
    gaus = lambda x, N, mu, sigma: N*np.exp(-(x-mu)**2/(2.0*sigma**2))
    coeff, _ = curve_fit(gaus, bin_centers, dt_hist, p0=[amplitude, peak_center, sigma])
    ax.plot(
        bin_centers,
        gaus(bin_centers, *coeff),
        color=colors[1],
        label=f'{label}\n {r"$\mu$"}: {coeff[1]:.3f}\n{r"$\sigma$"}: {abs(coeff[2]):.4f}',
    )

#[{'xcenter': 3.66994, 'ycenter': 0.8579705, 'width': 1.4452183, 'height': 1.3573316}]
row, col = 9,8
pix_sel_etroc = (dataPresel.row==row) & (dataPresel.col==col)

px_pos = find_pxl_from_traks(xhits[row, col], yhits[row, col])[0]
pix_sel_track = (
    (px_pos['xcenter']-1.3/2 <=dataPresel.x) & (dataPresel.x<=px_pos['xcenter']+1.3/2)
    &
    (px_pos['ycenter']-1.3/2 <=dataPresel.y) & (dataPresel.y<=px_pos['ycenter']+1.3/2)    
)

chi2_sel = (dataPresel.chi2 < 30) & (dataPresel.chi2 > 0)
bad_track = ((dataPresel['x'] != -1000) | (dataPresel['y'] != -1000))
px_matched_events = dataPresel[pix_sel_etroc & pix_sel_track & chi2_sel]

fig, axs = plt.subplots(1,2,  figsize=(14, 6))

# dt
axs[0].set_title(r"$\Delta$T")
left, right = 11, 13.5 #-10, 25 #
dts = ak.flatten(dataPresel['dt'][pix_sel_etroc])
dts_matched = ak.flatten(px_matched_events['dt'])
dt_plot(
    axs[0], 
    dts.to_numpy(),         
    np.linspace(left,right,150), 0.1, colors=('green', 'blue'), label=r"$\Delta$T")
dt_plot(
    axs[0], 
    dts_matched.to_numpy(), 
    np.linspace(left,right,75),  0.1, colors=('orange', 'red'), label=r"$\Delta$T Track Matched")
axs[0].set_xlabel("time (ns)")
axs[0].set_ylabel("counts")
axs[0].legend()

#TW Corrected dt
axs[1].set_title(r"$\Delta$T Timewalk Corrected")
left, right = -0.25, 0.25
dts = ak.flatten(dataPresel['dt_corr'][pix_sel_etroc])
dts_corr_matched = ak.flatten(px_matched_events['dt_corr'])
dt_plot(
    axs[1], 
    dts.to_numpy(),              
    np.linspace(left,right,90), 0.1, colors=('green', 'blue'), label=r"$\Delta$T")
dt_plot(
    axs[1], 
    dts_corr_matched.to_numpy(), 
    np.linspace(left,right,45), 0.1, colors=('orange', 'red'), label=r"$\Delta$T Track Matched")
axs[1].set_xlabel("time (ns)")
axs[1].set_ylabel("counts")
axs[1].legend()